In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Input,Dense,Conv2D,BatchNormalization
from tensorflow.keras.utils import plot_model
import tensorflow_datasets as tfds

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/tf-data/masterfile.csv')

img= data['0']

dir = '../input/tf-data/Final_data/'
images  = [dir + addr for addr in img ]

img_arr = []

for n, image in tqdm(enumerate(images), total=len(images)):

    image = cv2.imread(image)
    img_arr.append(image)
    


In [ ]:
img_arr = np.array(img_arr)
img_arr.shape

In [ ]:
data.drop(['0'],axis = 1 ,inplace = True)
Y_true = np.array(data)
#Y_true = Y_true/224.0
#Y_true = Y_true[:10000,:]
Y_true

In [ ]:
#img_arr = np.array(pd.read_csv('../input/sem-dataset/Dataset.csv'))
img_arr
def read_image_and_annotation(image, annotation):
    '''
    Casts the image and annotation to their expected data type and
    normalizes the input image so that each pixel is in the range [-1, 1]

    Args:
     image (numpy array) -- input image
     annotation (numpy array) -- ground truth label map

    Returns:
     preprocessed image-annotation pair
    '''

    image = tf.cast(image, dtype=tf.float32)
    #image = tf.reshape(image, (image.shape[0], image.shape[1], 1,))
    annotation = tf.cast(annotation, dtype=tf.int32)
    image = image/255
    

    return image, annotation


def get_training_dataset(images, annos):
    '''
    Prepares shuffled batches of the training set.
  
    Args:
      images (list of strings) -- paths to each image file in the train set
      annos (list of strings) -- paths to each label map in the train set

    Returns:
      tf Dataset containing the preprocessed train set
    '''
    training_dataset = tf.data.Dataset.from_tensor_slices((images, annos))
    training_dataset = training_dataset.map(read_image_and_annotation)

    training_dataset = training_dataset.shuffle(1024)
    training_dataset = training_dataset.batch(128)
    training_dataset = training_dataset.repeat()


    return training_dataset

train_dataset = get_training_dataset(img_arr,Y_true)

train_dataset

In [ ]:
inputs = tf.keras.layers.Input(shape = (224,224,3,))
b1 = tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3),padding = 'same',activation = 'relu')(inputs)
b1= tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b1)
b1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b1)

b2 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b1)
b2 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b2)
b2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b2)

b3 = tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b2)
b3 = tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
b3= tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
b3= tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b3)

b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
b4 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b4)

b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b5)
b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b5)
b5 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b5)

c6 = tf.keras.layers.Conv2D(4096,(7,7),activation  = 'relu',padding = 'same')(b5)
c7 = tf.keras.layers.Conv2D(4096,(1,1),activation  = 'relu',padding = 'same')(c6)
g1 = tf.keras.layers.GlobalAveragePooling2D()(c7)
d1= tf.keras.layers.Dense(128,activation = 'relu')(g1)
d2 = tf.keras.layers.Dense(2)(d1)
model = tf.keras.Model(inputs = inputs,outputs = d2)
model.compile(optimizer = 'adam',loss = 'mse',metrics = ['accuracy'])
model.summary()
#model.fit(images,Y,epochs = 30,callbacks=[tf.keras.callbacks.CSVLogger('training.csv')])

# def block(x,n_convs,filters,kernel_size,activation,pool_size,pool_stride,block_name):

#     for i in range(n_convs):
#         x = tf.keras.layers.Conv2D(filters = filters , kernel_size = kernel_size,padding = 'same',activation = activation,name = f'{block_name}_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=pool_size,strides = pool_stride,name = f'{block_name}_pool{i+1}')(x)
#     return x


# def main_model(image_inputs):

#     b0 = image_inputs
#     b0 = tf.image.grayscale_to_rgb(b0)

    
#     for i in range(2):
#         x = tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3),padding = 'same',activation = 'relu',name = f'block1_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2),name = f'block1_pool{i+1}')(x)
    
#     for i in range(2):
#         x = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3),padding = 'same',activation = 'relu',name = f'block2_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2),name = f'block2_pool{i+1}')(x)

#     for i in range(3):
#         x = tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu',name = f'block3_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2),name = f'block3_pool{i+1}')(x)

#     for i in range(3):
#         x = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu',name = f'block4_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2),name = f'block4_pool{i+1}')(x)
    
#     for i in range(3):
#         x = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu',name = f'block5_conv{i+1}')(x)

#     x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2),name = f'block5_pool{i+1}')(x)

    

#     model = tf.keras.Model(inputs = image_inputs , outputs = x)



#     # Number of filters in the output layer
  
#     n  =4096

#     c6 = tf.keras.layers.Conv2D(n,(7,7),activation  = 'relu',padding = 'same',name = 'conv6')(b5)

#     c7 = tf.keras.layers.Conv2D(n,(1,1),activation  = 'relu',padding = 'same',name=  'conv7')(c6)

#     g1 = tf.keras.layers.GlobalAveragePooling2D()(c7)

#     d1 = tf.keras.layers.Dense(128,activation = 'relu')(g1)

#     d2 = tf.keras.layers.Dense(2)(d1)

#     return d2


# def Coordinate_ext():

#     inputs = tf.keras.layers.Input(shape = (224,224,1,))

#     outputs  = main_model(inputs)
#     model = tf.keras.Model(inputs = inputs,outputs = outputs)

#     return model

# model = Coordinate_ext()
# model.summary()



In [ ]:
images =  img_arr[:10000,:,:,:]
images.shape
Y = Y_true[:10000,:]
Y.shape


In [ ]:
model.compile(optimizer = 'adam',loss = 'mse',metrics = ['accuracy'])
model.fit(images,Y,epochs = 10,steps_per_epoch = 128,callbacks=[tf.keras.callbacks.CSVLogger('training.csv')])

In [ ]:
model.save('model_huber.h5')

In [ ]:
def block(x,n_convs,filters,kernel_size,activation,pool_size,pool_stride,block_name):

    for i in range(n_convs):
        x = tf.keras.layers.Conv2D(filters = filters , kernel_size = kernel_size,padding = 'same',activation = activation,name = f'{block_name}_conv{i+1}')(x)

    x = tf.keras.layers.MaxPool2D(pool_size=pool_size,strides = pool_stride,name = f'{block_name}_pool{i+1}')(x)
    return x


In [ ]:

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
     # define your model normally
    inputs = tf.keras.layers.Input(shape = (224,224,3,))
    b1 = tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3),padding = 'same',activation = 'relu')(inputs)
    b1= tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b1)
    b1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b1)

    b2 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b1)
    b2 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b2)
    b2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b2)

    b3 = tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b2)
    b3 = tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
    b3= tf.keras.layers.Conv2D(filters = 256 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
    b3= tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b3)

    b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b3)
    b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
    b4 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
    b4 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b4)

    b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b4)
    b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b5)
    b5 = tf.keras.layers.Conv2D(filters = 512 , kernel_size = (3,3),padding = 'same',activation = 'relu')(b5)
    b5 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = (2,2))(b5)

    c6 = tf.keras.layers.Conv2D(4096,(7,7),activation  = 'relu',padding = 'same')(b5)
    c7 = tf.keras.layers.Conv2D(4096,(1,1),activation  = 'relu',padding = 'same')(c6)
    g1 = tf.keras.layers.GlobalAveragePooling2D()(c7)
    d1= tf.keras.layers.Dense(128,activation = 'relu')(g1)
    d2 = tf.keras.layers.Dense(2)(d1)
    model = tf.keras.Model(inputs = inputs,outputs = d2)
    model.compile(optimizer = 'adam',loss = 'mse',metrics = ['accuracy'])

# train model normally

model.fit(images,Y,epochs = 100,steps_per_epoch = 128,callbacks=[tf.keras.callbacks.CSVLogger('training.csv')])

In [ ]:
# img = cv2.imread('../input/tf-data/Final_data/002ae037be8b7b7a8605866296c2d0a1.png')
# img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.imread('../input/tf-data/Final_data/001d4373e64905a5804aa42a27bb8b83.png')

print(image.shape)
image = np.reshape(image,[1,224,224,3]) 
prediction = model.predict(image)
[[x,y]] = prediction

print(f'X : {x}  , Y : {y}')

In [ ]:
model.save('TPU_Trained_model.h5')